In [22]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.envrc")

True

In [23]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [24]:
import os
loader = CSVLoader(os.path.join(os.getcwd(),"cusinfo.csv"))

In [25]:
data = loader.load()

In [26]:
llm = OpenAI(temperature=0.7)
embeddings = OpenAIEmbeddings()

In [27]:
vectorstore = FAISS.from_documents(data, embeddings)

In [28]:
from langchain.chains import RetrievalQA

In [29]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), return_source_documents=True)

In [30]:
response = chain.invoke("How many rows are there in the dataset?")
print(response["result"])
print("Bassed on the Data:", response["source_documents"][0].metadata['source'])

There are 4 rows in the dataset.
Bassed on the Data: /Users/rajath/analogdata/spaacesai/research/cusinfo.csv


In [31]:
chain.invoke("How many visits are scheduled for tomorrow that is 23-05-2024 ?")

{'query': 'How many visits are scheduled for tomorrow that is 23-05-2024 ?',
 'result': ' 1',
 'source_documents': [Document(page_content='customer_id: 1\nname: Rajath Kumar\nemail: rajathkumarks@gmail.com\nphone: 9108399333\nvisit_planned: 23-05-2024', metadata={'source': '/Users/rajath/analogdata/spaacesai/research/cusinfo.csv', 'row': 0}),
  Document(page_content='customer_id: 4\nname: Vijay\nemail: vijay@elementure.tech\nphone: 9108399555\nvisit_planned: 27-05-2024', metadata={'source': '/Users/rajath/analogdata/spaacesai/research/cusinfo.csv', 'row': 3}),
  Document(page_content='customer_id: 3\nname: Vikram\nemail: vikram@elementure.tech\nphone: 9108399333\nvisit_planned: 26-05-2024', metadata={'source': '/Users/rajath/analogdata/spaacesai/research/cusinfo.csv', 'row': 2}),
  Document(page_content='customer_id: 5\nname: Pushpa\nemail: pushpa@elementure.tech\nphone: 9108399888\nvisit_planned: 30-05+2024', metadata={'source': '/Users/rajath/analogdata/spaacesai/research/cusinfo.csv

### HTML Loader

In [32]:
from langchain_community.document_loaders import BSHTMLLoader

In [33]:
web_file = os.path.join(os.getcwd(),"web_mspaaces","engrace_vista.html")

In [34]:
web_loader = BSHTMLLoader(os.path.join(os.getcwd(),"web_mspaaces","engrace_vista.html"))

In [35]:
web_data = web_loader.load()

In [36]:
from langchain_text_splitters import HTMLSectionSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [37]:
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("h5", "Header 5"),
    ("h6", "Header 6"),
    ("p", "Paragraph"),
]

In [38]:
website_data = open(web_file, "r").read()

In [39]:
html_splitter = HTMLSectionSplitter(headers_to_split_on)

html_header_splits = html_splitter.split_text(website_data)

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

In [40]:
splits = text_splitter.split_documents(html_header_splits)

In [41]:
splits

[Document(page_content='You need to enable JavaScript to run this app.\n   \n \n \n \n \n \n \n \n \n \n \n \n \n \n            Projects\n            \n \n \n \n \n \n \n \n \n             Ongoing Projects\n             \n \n \n \n \n \n               Soulace\n               \n \n \n \n               Engrace\n               \n \n               4 Units Left\n               \n \n \n \n               Engrace Vista\n               \n \n \n \n \n \n             Upcoming Projects\n             \n \n \n \n \n              Central Park\n              \n \n              Serene Heights', metadata={'Header 1': '#TITLE#'}),
 Document(page_content='Serene Heights\n              \n \n              Onyx\n              \n \n              Green Storeys\n              \n \n              Windmere\n              \n \n              Walk Town Residences\n              \n \n              Tranquil Terraces\n              \n \n              Ivy County\n              \n \n              Fo’Rest of ur lyf\n      

In [42]:
vectorstore = FAISS.from_documents(splits, embeddings)

In [43]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [44]:
chain.invoke("When is the Possession Date ?")

{'query': 'When is the Possession Date ?', 'result': ' June 2027 onwards'}

In [45]:
chain.invoke("What is the construction technology used")

{'query': 'What is the construction technology used',
 'result': ' Aluform Technology'}

In [46]:
chain.invoke("What are the legal compliance available, Is it RERA Approved ?")

{'query': 'What are the legal compliance available, Is it RERA Approved ?',
 'result': ' Yes, the legal compliance available is RERA Approved.'}

In [47]:
chain.invoke("Can you tell the location of engrace vista")

{'query': 'Can you tell the location of engrace vista',
 'result': ' Engrace Vista is located in the Sompura Gate area of Sarjapur, Bangalore.'}

In [48]:
chain.invoke("What are the education centers near by ?")

{'query': 'What are the education centers near by ?',
 'result': ' Some of the education centers near the Engrace Vista community include Indus International School, Bangalore, and various other premier schools and colleges located within a convenient 5 to 8 kilometers from the community.'}

In [49]:
chain.invoke("Are there any shopping mall near by ?")

{'query': 'Are there any shopping mall near by ?',
 'result': ' Yes, there are shopping centers, restaurants, eateries, and brew pubs within a 5-10 minute drive.'}

In [50]:
chain.invoke("Can i get the phone number of the modern spaaces")

{'query': 'Can i get the phone number of the modern spaaces',
 'result': ' The phone number for Modern Spaaces is +91 9008 288 288.'}

In [51]:
chain.invoke("Why I need to purchase from modern spaaces")

{'query': 'Why I need to purchase from modern spaaces',
 'result': ' It is recommended to purchase from modern spaaces as it is the only official website and they have a proven track record of success in creating innovative and high-quality spaces. Additionally, their focus on using technology, design, and natural elements ensures that their spaces are not only aesthetically pleasing, but also enhance the quality of life for their customers.'}

In [52]:
chain.invoke("Is modern spaaces is RERA Approved")

{'query': 'Is modern spaaces is RERA Approved',
 'result': "\nI don't know, as it is not mentioned in the given context. "}

In [54]:
chain.invoke("What is the email id of modern spaaces")

{'query': 'What is the email id of modern spaaces',
 'result': ' info@modernspaaces.com'}

In [55]:
chain.invoke("Is it EV Charging facility available")

{'query': 'Is it EV Charging facility available',
 'result': ' Yes, EV charging facility is available at dedicated parking spaces.'}

In [56]:
chain.invoke("Is modern spaaces offer smart water meters ?")

{'query': 'Is modern spaaces offer smart water meters ?',
 'result': ' It is not explicitly mentioned in the context, so it is not clear if Modern Spaaces offers smart water meters.'}

In [57]:
chain.invoke("Is lift / elevators will be installed ?")

{'query': 'Is lift / elevators will be installed ?',
 'result': ' Yes, elevators will be installed.'}

In [58]:
chain.invoke("Can you give more information about elevators that are going to be installed")

{'query': 'Can you give more information about elevators that are going to be installed',
 'result': ' The elevators will have automatic rescue devices and an emergency call facility to security. Each tower will have three lifts, with two designated for residents and one for service. This will improve mobility management and reduce wait times for elevators.'}

In [59]:
chain.invoke("How spacious apartments are available in modern spaaces")

{'query': 'How spacious apartments are available in modern spaaces',
 'result': ' \n\nThe context does not provide a specific number of apartments available. It only mentions that the apartments have floor areas ranging from 2116 - 2200 sq.ft, which suggests that there are multiple apartments available. '}

In [60]:
chain.invoke("Create a pitch for modern spaaces engrace vista")

{'query': 'Create a pitch for modern spaaces engrace vista',
 'result': " Modern Spaaces is a company that has been creating incredible spaces that blend technology, design, and natural elements to enhance people's lives. Our latest project, Engrace Vista, is a testament to our commitment to innovation and excellence. Located in the beautiful setting of Soulace, Engrace Vista offers a unique living experience where you can truly connect with nature. With upcoming projects like Central Park, Serene Heights, and Onyx, we are taking sustainable living to new heights. Don't miss the opportunity to be a part of this extraordinary development. Explore Engrace Vista today and access all the project resources, including floor plans and master plans, to see the intricate details that make this project truly special. Join us in creating a better world, one space at a time."}